## Part 1: Web-scrapping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

***Setting Maximum column width for visualization***

In [2]:
pd.set_option('max_colwidth', 800)

***Getting the source webpage and assigining the variable source to it and iniatilizing the beautifulsoup object to soup***

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

In [4]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

***Scraping the page to extract the data table***

In [5]:
table = soup.find('table', class_ = 'wikitable sortable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned\n':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned\n': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

In [6]:
csv_file.close()

In [7]:
df = pd.read_csv('toronto_postal_codes.csv')

In [8]:
# Removing '\n' from the end

df = df.replace('\n','', regex=True)

***Getting the shape of the dataframe***

In [9]:
df.shape

(103, 3)

In [10]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Part 2: Latitude and Longitude

In [11]:
postal_codes = df['Postcode'].values

***Using the OpenCage GeoCoder API***

In [12]:
import json

latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for postal_code in postal_codes : 
    place_name = postal_code + " Toronto" # Formats the place name
    url = 'https://api.opencagedata.com/geocode/v1/json?q={}&key=77d6d969c7914004ae810639e74a41cd'.format(place_name) # Gets the proper url to make the API call
    obj = json.loads(requests.get(url).text) # Loads the JSON file in the form of a python dictionary
    
    results = obj['results'] # Extracts the results information out of the JSON file
    lat = results[0]['geometry']['lat'] # Extracts the latitude value
    lng = results[0]['geometry']['lng'] # Extracts the longitude value
    
    latitudes.append(lat) # Appending to the list of latitudes
    longitudes.append(lng) # Appending to the list of longitudes

***Adding the latitude and longitude values to the dataframe***

In [13]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [14]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,-33.013441,151.594204
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-33.013441,151.594204


## Part 3: FourSquare API

In [15]:
# Downloading folium, if not installed
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

In [16]:
CLIENT_ID = 'WQXS5RMB24A5EVJC05UDBIA3XXJA2JBTUAR2KEZJUJD2D5K5' 
CLIENT_SECRET = 'CJOAPMRW3LG4UCHSZIJWZU2Y42UOMQ2DF14I05ONO2UYU5CN'
VERSION = '20180604'

***Creating a folium map of Toronto***

In [17]:
# Toronto latitude and longitude using Google search
tor_lat = 43.6532
tor_lng = -79.3832

# Creates map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lng], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

***Defining own function to get the name of the category***

In [18]:
# Gets the name of the category

def get_category_type(row):
    try:
        categories_list = row['Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

***Using the FourSquare API on all neighbourhoods***

In [19]:
explore_df_list = []

for i, nbd_name in enumerate(df['Neighbourhood']):  
    
    try :         
        ### Getting the data of neighbourhood
        nbd_name = df.loc[i, 'Neighbourhood']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 500 # Setting the radius as 500 metres
        LIMIT = 100 # Getting the top 100 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            explore_df_list.append([nbd_name, nbd_lat, nbd_lng] + nearby.loc[i, :].values.tolist())
    
    except Exception as e:
        pass

***Creating a dataframe for performing clustering operations***

In [20]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Parkwoods,-33.013441,151.594204,Subway,Sandwich Place,-33.013180,151.592120
1,Parkwoods,-33.013441,151.594204,Woolworths,Supermarket,-33.014511,151.595918
2,Parkwoods,-33.013441,151.594204,Doubletake Cafe,Café,-33.014280,151.597680
3,Parkwoods,-33.013441,151.594204,Coles,Supermarket,-33.012820,151.595456
4,Parkwoods,-33.013441,151.594204,Michel's Patisserie,Coffee Shop,-33.012720,151.595440


***Total number of unique categories***

In [21]:
len(explore_df['Venue Category'].unique())

254

***Performing one-hot encoding to analyze neighbourhoods***

In [22]:
# One hot encoding
toronto_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = explore_df['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


***Aggregating venues by neighbourhoods***

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Agincourt,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***Creating a function to return the most common venues***

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

***Creating a new dataframe to get the top 10 venues***

In [25]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Badminton Court,Breakfast Spot,Latin American Restaurant,Skating Rink,Yoga Studio,Flower Shop,Field,Filipino Restaurant,Fish Market,Flea Market
1,"Alderwood, Long Branch",Pizza Place,Convenience Store,Sandwich Place,Gym,Coffee Shop,Pub,Flea Market,Fish Market,Filipino Restaurant,Flower Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Fried Chicken Joint,Coffee Shop,Mediterranean Restaurant,Middle Eastern Restaurant,Grocery Store,Pizza Place,Deli / Bodega,Diner,Dance Studio,Food Truck
3,Bayview Village,Flower Shop,Dog Run,Trail,Gas Station,Park,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Liquor Store,Pub,Fast Food Restaurant,Comfort Food Restaurant,Thai Restaurant,Juice Bar


***Using K-Means clustering***

In [26]:
# Set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

***Creating a grand dataframe containing the necessary data***

In [27]:
toronto_merged = df
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,-33.013441,151.594204,4,Café,Supermarket,Coffee Shop,Sandwich Place,Fast Food Restaurant,Waterfront,Harbor / Marina,Filipino Restaurant,Farmers Market,Field
1,M4A,North York,Victoria Village,43.727600,-79.314800,4,French Restaurant,Intersection,Portuguese Restaurant,Park,Coffee Shop,Pizza Place,Hockey Arena,Hobby Shop,Food,Flower Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600,4,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Spa,Event Space,Restaurant,Beer Store,Electronics Store,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400,4,Clothing Store,Coffee Shop,Women's Store,Restaurant,Bakery,Shoe Store,Sandwich Place,Jewelry Store,Men's Store,Cosmetics Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-33.013441,151.594204,4,Café,Supermarket,Coffee Shop,Sandwich Place,Fast Food Restaurant,Waterfront,Harbor / Marina,Filipino Restaurant,Farmers Market,Field


***Visualizing the clusters***

In [28]:
# Create map
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

***Cluster 0***

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,"Islington Avenue, Humber Valley Village",0,Pharmacy,Bank,Skating Rink,Grocery Store,Park,Home Service,Food & Drink Shop,Food,Flower Shop,Flea Market
21,York,Caledonia-Fairbanks,0,Park,Bakery,Mexican Restaurant,Beer Store,Gym,Sporting Goods Shop,Women's Store,Diner,Fast Food Restaurant,Fountain
22,Scarborough,Woburn,0,Park,Indian Restaurant,Taco Place,Coffee Shop,Food Court,Food & Drink Shop,Food,Flower Shop,Food Truck,Ethiopian Restaurant
27,North York,Hillcrest Village,0,Park,Residential Building (Apartment / Condo),Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Yoga Studio,Ethiopian Restaurant
32,Scarborough,Scarborough Village,0,Grocery Store,Park,Yoga Studio,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
35,East York,"East Toronto, Broadview North (Old East York)",0,Park,Massage Studio,Convenience Store,Coffee Shop,Food Truck,Food Court,Fountain,Food & Drink Shop,Food,Event Space
39,North York,Bayview Village,0,Flower Shop,Dog Run,Trail,Gas Station,Park,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market
61,Central Toronto,Lawrence Park,0,Park,Photography Studio,Yoga Studio,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop
64,York,Weston,0,Home Service,Convenience Store,Park,Yoga Studio,Fish Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flower Shop
66,North York,York Mills West,0,Convenience Store,Park,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flower Shop


***Cluster 1***

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,Berczy Park,1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
30,Downtown Toronto,"Richmond, Adelaide, King",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
33,North York,"Fairview, Henry Farm, Oriole",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
36,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
43,West Toronto,"Brockton, Parkdale Village, Exhibition Place",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
59,North York,"Willowdale, Willowdale East",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
63,York,"Runnymede, The Junction North",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
69,West Toronto,"High Park, The Junction South",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
81,West Toronto,"Runnymede, Swansea",1,Home Service,Lawyer,Yoga Studio,Event Space,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


***Cluster 2***

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,"Wexford, Maryvale",2,Auto Garage,Yoga Studio,Event Space,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop


***Cluster 3***

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,"York Mills, Silver Hills",3,Pool,Yoga Studio,Ethiopian Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


***Cluster 4***

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,4,Café,Supermarket,Coffee Shop,Sandwich Place,Fast Food Restaurant,Waterfront,Harbor / Marina,Filipino Restaurant,Farmers Market,Field
1,North York,Victoria Village,4,French Restaurant,Intersection,Portuguese Restaurant,Park,Coffee Shop,Pizza Place,Hockey Arena,Hobby Shop,Food,Flower Shop
2,Downtown Toronto,"Regent Park, Harbourfront",4,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Spa,Event Space,Restaurant,Beer Store,Electronics Store,Food Truck
3,North York,"Lawrence Manor, Lawrence Heights",4,Clothing Store,Coffee Shop,Women's Store,Restaurant,Bakery,Shoe Store,Sandwich Place,Jewelry Store,Men's Store,Cosmetics Shop
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",4,Café,Supermarket,Coffee Shop,Sandwich Place,Fast Food Restaurant,Waterfront,Harbor / Marina,Filipino Restaurant,Farmers Market,Field
6,Scarborough,"Malvern, Rouge",4,Café,Supermarket,Coffee Shop,Sandwich Place,Fast Food Restaurant,Waterfront,Harbor / Marina,Filipino Restaurant,Farmers Market,Field
7,North York,Don Mills,4,Restaurant,Pizza Place,Coffee Shop,Bank,Yoga Studio,Jewelry Store,Liquor Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
8,East York,"Parkview Hill, Woodbine Gardens",4,Pizza Place,Intersection,Bank,Gym / Fitness Center,Gastropub,Breakfast Spot,Pharmacy,Café,Pet Store,Food Court
9,Downtown Toronto,"Garden District, Ryerson",4,Coffee Shop,Clothing Store,Café,Hotel,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Furniture / Home Store,Pizza Place,Bubble Tea Shop
10,North York,Glencairn,4,Pizza Place,Italian Restaurant,Mediterranean Restaurant,Fish Market,Gas Station,Fast Food Restaurant,Latin American Restaurant,Rental Car Location,Japanese Restaurant,Grocery Store
